In [1]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import os

import torch
import pandas as pd
from sklearn.model_selection import train_test_split

# Clase para cargar el tokenizador
from transformers import AutoTokenizer
# Clase para secuencias a clasificación (N -> 1)
from transformers import AutoModelForSequenceClassification

# clases para entrenamiento
from transformers import Trainer, TrainingArguments

# Numpy
import numpy as np

# operative system
import os


In [2]:
#DATA_FOLDER="../data/economicos/data_cruda"
#df = pd.concat([pd.read_json(f"{DATA_FOLDER}/{imgfile}") for imgfile in os.listdir(DATA_FOLDER)])
#df.to_parquet("../data/economicos/join.parquet")
DATA_FILE = "../data/economicos/join.parquet"

In [3]:
df = pd.read_parquet(DATA_FILE)
df = df.rename(columns = {
    'url': 'url',
    'description': 'description',
    'price': 'price',
    'title': 'title',
    'address': 'address',
    'images': 'images',
    'Tipo:': 'type',
    'Operación:': 'operation',
    'm  construidos:': 'm_built',
    'm  terreno:': 'm_size',
    'Región:': 'state',
    'Comuna:': 'county',
    'Fecha Publicación:': 'date',
    'Diario:': 'source',
    'Dormitorios:':'rooms',
    'Baños:': 'bathrooms'
})


In [4]:
filter = ((df.state == 'Metropolitana de Santiago') & (df['type'].isin(['Departamento', 'Casa', 'Departamento Amoblado'])) & (df['operation'].isin(['Arriendo', 'Venta'])) )
columns = ['description', 'price', 'm_built', 'm_size', 'county', 'date', 'rooms', 'bathrooms', 'title', 'url', 'operation']
stgo = df.loc[filter, columns].drop_duplicates().reset_index(drop=True)
display(stgo.shape[0])

241658

In [5]:
display(stgo.description.iloc[0])
display(stgo.rooms.iloc[0])

'\nArriendo en Diagonal Paraguay 194, piso 14o, Departamento Amoblado de 2 dormitorios, 2 Baños, Living- Comedor con Balcón, Cocina, con instalación de Lavadora, Aire Acondicionado, Internet. Todos los Muebles se entregan en custodia al arrendatario. Piso Flotante, Cerámicas Baños Y Cocina, renta de arrendamiento $450.000 mensuales, Gasto Común $ 60.000 Aprox. Orientación Sur. Disponible para ser habitado, Contacto Ninette Martínez A, Celular 9 8669 7611, El edificio cuenta con Sala de Eventos, Gimnasio, lavandería, Entorno: Facultad de Arquitectura y Economía, Hospital Universidad Católica, Supermercados. Farmacias Metro Bustamante, Baquedano, U. Católica, Buses, Taxis. Contactar al 9 8669 7611, para Agendar Visita\n\n'

2.0

In [6]:
display(stgo[(stgo.rooms<1) | (stgo.rooms>6)].shape[0])
stgo_rooms = stgo[(stgo.rooms>=1) & (stgo.rooms<=6)]
display(stgo_rooms.shape[0])

1977

226375

In [7]:
display(stgo_rooms.rooms.agg(['min', 'max']))
stgo_rooms['n_rooms'] = (stgo_rooms.rooms-stgo_rooms.rooms.min())/(stgo_rooms.rooms.max()-stgo_rooms.rooms.min())

display(stgo_rooms.n_rooms.agg(['min', 'max']))

min    1.0
max    6.0
Name: rooms, dtype: float64

/tmp/ipykernel_24434/383088303.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stgo_rooms['n_rooms'] = (stgo_rooms.rooms-stgo_rooms.rooms.min())/(stgo_rooms.rooms.max()-stgo_rooms.rooms.min())


min    0.0
max    1.0
Name: n_rooms, dtype: float64

In [8]:
# Selecciono el modelo base y creo un tokenizador para realizar pruebas
model_name = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Imprimo el tamaño de la vocabulario
display(tokenizer.vocab_size)
# Imprimo los token especiales
display(tokenizer.special_tokens_map)

31002

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [9]:
# constante para entrenar o no
TRAIN_MODE = False
MODEL_PATH="best_rooms/checkpoint-45846/pytorch_model.bin"
#SNAPSHOT="results/checkpoint-45846/pytorch_model.bin"
SNAPSHOT=None
TRAIN_EPOCHS=20
# Cuda si existe grafica, cpu si no
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
# Carga de datasets
df_train, df_test = train_test_split(stgo_rooms, test_size=0.1, stratify=stgo_rooms.rooms)

In [11]:
# reseteo el index, importante para crear los datasets
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [12]:
df_train.description.map(len).max()

13754

In [13]:
# Creo el dataset, los minimos elementos a implementar son
# __init__, __len__ y __getitem__
# esto es porque itera con un for simple
class PandasDataset(torch.utils.data.Dataset):
  def __init__(self, df, x, y, tokenizer):    
    self.x = df[x]
    self.y = df[y]
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.x)

  def __getitem__(self, ix):
    return {
                **self.tokenizer(self.x[ix], truncation=True, padding="max_length", max_length=300),
                **{"label": self.y[ix], "text": self.x[ix]}
            }

In [14]:
# Creo el modelo para clasificación, para el numero de label de nuestro problema
# model_name es el nombre del modelo que quiero usar como base
num_labels = 1
model = (AutoModelForSequenceClassification
         .from_pretrained(model_name, num_labels=num_labels)
         .to(device))
if SNAPSHOT:
    model.load_state_dict(torch.load(SNAPSHOT))

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
# Determino los parametros del entrenamiento
BATCH_SIZE = 40
logging_steps = len(df_train) // BATCH_SIZE
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=TRAIN_EPOCHS,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=BATCH_SIZE,
                                  per_device_eval_batch_size=BATCH_SIZE,
                                  load_best_model_at_end=True,
                                  #metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,)

In [ ]:
trainer = Trainer(model=model, args=training_args, train_dataset=PandasDataset(df_train, "description", "rooms", tokenizer), eval_dataset=PandasDataset(df_test, "description", "rooms", tokenizer))
if TRAIN_MODE:
  trainer.train();

results/checkpoint-4284

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

# Dataset parecido al de entrenamiento, pero dedicado a predecir
class PandasDatasetTest(torch.utils.data.Dataset):
  def __init__(self, df, x, y, tokenizer):    
    self.x = df[x]
    self.y = df[y]
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.x)

  def __getitem__(self, ix):
    data = self.tokenizer(self.x[ix], truncation=True, padding="max_length", max_length=512)
    return data["input_ids"], data["attention_mask"]

test_dataloader = torch.utils.data.DataLoader(PandasDatasetTest(df_test, "description", "rooms", tokenizer), batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [ ]:
history_models = {}

if not TRAIN_MODE:
  model.load_state_dict(torch.load(MODEL_PATH))

# Dejo al modelo en modo evaluación (evita calculos innecesarios)
model.eval()

# donde guardo los resultados
s_results = []
# itero por los batches del dataset de test
for item in test_dataloader:
  # Obtengo el batch de input_ids dimencion (B, 23, )
  # 23 es el largo debido al padding
  input_ids = torch.tensor(item[0]).to(device)
  # Obtengo la mascara usada (B, 23, )
  attention_mask = torch.tensor(item[1]).to(device)
  # Obtengo el resultado 
  result = model(input_ids, attention_mask=attention_mask)
  s_results.append([float(x) for x in result.logits])
  # Borro los elemetos, importante para liberar memoria
  del input_ids
  del attention_mask
  del result



In [ ]:
df = df_test

df["y_pred"] = [round(x) for x in np.concatenate([np.array(i) for i in s_results])]
df["y_pred_float"] = np.concatenate([np.array(i) for i in s_results])
# Muestro los que no fueron bien asignados
x_labels = list(range(1,8,1))
y_labels = list(range(1,8,1))
matrix = np.zeros((len(x_labels), len(y_labels)))
for x in x_labels:
  for y in y_labels: 
    matrix[x-1,y-1] = df.loc[(df.rooms == x) & (round(df.y_pred_float) == y)].shape[0]
# Muestro la matrix
display(pd.DataFrame(matrix, index=x_labels, columns=y_labels))

In [ ]:
# Sección de pruebas
record = df_test[(df_test.rooms != df_test.y_pred_float.round())].iloc[6]
TEXTO_PRUEBA=record.description
# Obtengo el vector y la mascara de uso
input_ids, _, attention_mask = tokenizer(TEXTO_PRUEBA).values()
# Paso a memoria de la grafica
input_ids, attention_mask = torch.tensor(input_ids).unsqueeze(0).to(device), torch.tensor(attention_mask).unsqueeze(0).to(device)
# Calculo
result = model(input_ids, attention_mask=attention_mask)

print("="*30)
display(record.description)
display(record.url)
print("="*30)
display(result)
print(f"y={record.rooms}")
print(f"y_pred={record.y_pred_float}")
print("="*30)
del result
del input_ids
del attention_mask

In [ ]:
df_test[(df_test.rooms == df_test.y_pred_float.round())].shape[0] / df_test.shape[0]

In [ ]:
df_test[(df_test.rooms != df_test.y_pred)].to_csv("no_match.csv", index=False)
df_test.loc[(df_test.rooms != df_test.y_pred_float.round()), ['description', 'rooms', 'y_pred', 'y_pred_float']]

In [ ]:
df_test[(df_test.rooms != df_test.y_pred_float.round())].iloc[0].description